In [1]:
from scraper import *

# 机构

In [5]:
%%time
df_companies = Company().get_company_list()

表格共有12列，121行。


,序号,机构名称,从业人员数,一般证券业务,证券投资咨询业务(其他),证券经纪业务营销,证券经纪人,证券投资咨询业务(分析师),证券投资咨询业务(投资顾问),保荐代表人,投资主办人,机构ID
0,1,爱建证券有限责任公司,1326,709,0,3,538,7,52,7,10,1999082
1,2,安信证券股份有限公司,8924,6081,0,190,1588,74,883,97,11,1999145
2,3,北京高华证券有限责任公司,135,98,0,0,0,16,21,0,0,1999142
3,4,渤海汇金证券资产管理有限公司,147,122,0,0,0,0,0,0,25,609
4,5,渤海证券股份有限公司,1796,1263,0,33,330,24,134,12,0,1999115
5,6,财达证券股份有限公司,2351,1662,0,14,467,22,161,6,19,1999040
6,7,财富证券有限责任公司,2735,1382,0,25,968,15,321,7,17,1999053
7,8,财通证券股份有限公司,3587,2278,0,2,840,35,393,39,0,1999016
8,9,长城国瑞证券有限公司,742,556,0,3,124,4,28,11,16,1999020
9,10,长城证券股份有限公司,4076,2289,0,1,1316,31,344,57,38,1999107


Wall time: 23.5 s


In [ ]:
df_companies.to_csv('列表_证券公司.csv', encoding='gb18030')

# 人员列表

In [2]:
# 单个公司人员列表：不含保存到本地的功能
%%time
df_people = People().get_people_list(company_ID='1999152')

100%|██████████| 2/2 [00:04<00:00,  2.14s/it]


表格共有11列，242行。


,PPP_ID,姓名,性别,学历,证书编号,执业机构,执业岗位,证书取得日期,证书有效截止日期,注册变更记录,诚信记录
0,76B6EA98CC523C0DE053D651A8C06CD1,牛岗,男,硕士研究生,S1520712100005,中德证券有限责任公司,保荐代表人,2012-10-19,2018-12-31,3条,无
1,76B6EA98D01A3C0DE053D651A8C06CD1,崔学良,男,硕士研究生,S1520712100002,中德证券有限责任公司,保荐代表人,2012-10-19,2018-12-31,3条,无
2,76B6EA98D01B3C0DE053D651A8C06CD1,单晓蔚,女,本科,S1520712100003,中德证券有限责任公司,保荐代表人,2012-10-19,2018-12-31,4条,无
3,76B6EA98D85F3C0DE053D651A8C06CD1,韩正奎,男,硕士研究生,S1520712100001,中德证券有限责任公司,保荐代表人,2012-10-19,2019-12-31,5条,无
4,76B6EA98DC393C0DE053D651A8C06CD1,李铁,男,博士研究生,S1520109070003,中德证券有限责任公司,一般证券业务,2009-07-22,2018-12-31,4条,无
5,76B6EA98DA863C0DE053D651A8C06CD1,梁炜,男,硕士研究生,S1520712100007,中德证券有限责任公司,保荐代表人,2012-10-19,2019-12-31,5条,无
6,76B6EA98E47A3C0DE053D651A8C06CD1,毛传武,男,本科,S1520712100009,中德证券有限责任公司,保荐代表人,2012-10-19,2018-12-31,4条,无
7,76B6EA98DD7E3C0DE053D651A8C06CD1,顾蓓菁,女,本科,S1520109110103,中德证券有限责任公司,一般证券业务,2009-11-12,2019-12-31,3条,无
8,76B6EA98E7933C0DE053D651A8C06CD1,万新,女,硕士研究生,S1520110020001,中德证券有限责任公司,一般证券业务,2010-02-03,2018-12-31,2条,无
9,76B6EA98EB3E3C0DE053D651A8C06CD1,陈亚东,男,硕士研究生,S1520712100010,中德证券有限责任公司,保荐代表人,2012-10-19,2019-12-31,4条,无


In [ ]:
# 所有公司人员列表：默认自动保存到本地
%%time
df_people_full = People().get_people_list_full(save_to_local=True)

# 个人信息
只涉及
1. 注册变更记录
2. 照片路径

In [4]:
%%time
People().get_person_info_img('76B6EA9DB8E33C0DE053D651A8C06CD1')

Wall time: 6.37 s


'http://person.sac.net.cn/photo/images/20080101/registrationRpInfo/136385645860216423.jpg'

# 结果保存到数据库